### Part 1: Data Preprocessing:
1.1 Load the dataset and perform initial exploration to understand its structure.

In [3]:
import pandas as pd

# Read the CSV file
df = pd.read_csv('News_Category_Dataset_v3.csv')

print(df.head())

   Unnamed: 0                                           headline   category  \
0           0  Over 4 Million Americans Roll Up Sleeves For O...  U.S. NEWS   
1           1  American Airlines Flyer Charged, Banned For Li...  U.S. NEWS   
2           2  23 Of The Funniest Tweets About Cats And Dogs ...     COMEDY   
3           3  The Funniest Tweets From Parents This Week (Se...  PARENTING   
4           4  Woman Who Called Cops On Black Bird-Watcher Lo...  U.S. NEWS   

                                   short_description               authors  \
0  Health experts said it is too early to predict...  Carla K. Johnson, AP   
1  He was subdued by passengers and crew when he ...        Mary Papenfuss   
2  "Until you have a dog you don't understand wha...         Elyse Wanshel   
3  "Accidentally put grown-up toothpaste on my to...      Caroline Bologna   
4  Amy Cooper accused investment firm Franklin Te...        Nina Golgowski   

         date  headline_length  short_description_length

1.2 Clean the text data, including removing special characters, stopwords, applying lowercasing, correcting spelling, standardizing, handling contractions, and lemtization.

In [4]:
from nltk.corpus import stopwords
import nltk
import re
from symspellpy import SymSpell, Verbosity
from nltk.stem import WordNetLemmatizer
import pkg_resources
import inflect
import contractions
nltk.download('stopwords')
nltk.download('wordnet')

stop_words = set(stopwords.words('english'))
sym_spell = SymSpell(max_dictionary_edit_distance=2, prefix_length=7)
dictionary_path = pkg_resources.resource_filename("symspellpy", "frequency_dictionary_en_82_765.txt")
bigram_path = pkg_resources.resource_filename("symspellpy", "frequency_bigramdictionary_en_243_342.txt")
sym_spell.load_dictionary(dictionary_path, term_index=0, count_index=1)
sym_spell.load_bigram_dictionary(bigram_path, term_index=0, count_index=2)
lemmatizer = WordNetLemmatizer()
p = inflect.engine()

def standardize_numbers(text):
    return ' '.join([p.number_to_words(word) if word.isdigit() else word for word in text.split()])

def handle_contractions(text):
    return contractions.fix(text)

def clean_text(text):
    text = str(text)
    # lowercase
    text = text.lower()
    # standardize
    text = standardize_numbers(text)
    # handle contractions
    text = handle_contractions(text)
    # correct typos
    words = text.split()
    corrected_words = []
    for word in words:
        suggestions = sym_spell.lookup(word, Verbosity.CLOSEST, max_edit_distance=2, include_unknown=True)
        corrected_words.append(suggestions[0].term if suggestions else word)
    text = ' '.join(corrected_words)
    # remove special characters
    text = re.sub(r'[^a-zA-Z0-9\s]', ' ', text)
    # remove stopwords
    words = [word for word in text.split() if word not in stop_words]
    # lemmatization
    words = [lemmatizer.lemmatize(word) for word in words]
    # rejoin words
    text = ' '.join(words)
    return text

# clean data
df['cleaned_headline'] = df['headline'].apply(clean_text)
df['cleaned_description'] = df['short_description'].apply(clean_text)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/muyangcheng329/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/muyangcheng329/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


1.3 Perform text tokenization and vectorization using TF-IDF.

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack, csr_matrix

tfidf_vectorizer = TfidfVectorizer()

tfidf_headline = tfidf_vectorizer.fit_transform(df['cleaned_headline'])
# df_headline_tfidf = pd.DataFrame(tfidf_headline.toarray(), columns=tfidf_vectorizer.get_feature_names_out())
# df_headline_tfidf = df_headline_tfidf.add_prefix('headline_')

tfidf_description = tfidf_vectorizer.fit_transform(df['cleaned_description'])
# df_description_tfidf = pd.DataFrame(tfidf_description.toarray(), columns=tfidf_vectorizer.get_feature_names_out())
# df_description_tfidf = df_description_tfidf.add_prefix('description_')

# df = pd.concat([df, df_headline_tfidf, df_description_tfidf], axis=1)
tfidf= hstack([tfidf_headline,tfidf_description])

1.4 Extract and analyze different features from the text that might be useful for classification, such as word count,
sentence length, n-grams, etc

In [7]:
#pip install category_encoders

from category_encoders import BinaryEncoder

# change date
df['date'] = pd.to_datetime(df['date'])
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day

# word count
df['headline_word_count'] = df['headline'].apply(lambda x: len(str(x).split()))
df['description_word_count'] = df['short_description'].apply(lambda x: len(str(x).split()))

# encode authors using Binary encoding
encoder = BinaryEncoder(cols=['authors'], return_df=True)
df_encoded = encoder.fit_transform(df['authors'])
df_encoded_sparse = csr_matrix(df_encoded.values)

# drop extra columns
selected_columns = ['year', 'month', 'day', 'headline_length', 'short_description_length', 'headline_word_count', 'description_word_count' ]
new_df = df[selected_columns].copy()
# combine
data = hstack([csr_matrix(new_df), df_encoded_sparse,tfidf])

# print(data)

#Convolutional Neural Network (CNN): Implement a CNN to capture local dependencies in text data. Discuss the choice of the architecture in the context of text data.

from sklearn.preprocessing import LabelEncoder

# Encode category labels with LabelEncoder
label_encoder = LabelEncoder()
df['category_encoded'] = label_encoder.fit_transform(df['category'])

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# We will use 'cleaned_description' as the input feature
texts = df['cleaned_description'].astype(str)

# Tokenize and encode the texts
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

# Use pad_sequences to ensure uniform input sequence length
data_padded = pad_sequences(sequences, maxlen=100)

# Prepare labels
labels = np.array(df['category_encoded'])


from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense

# Define the CNN model structure
model = Sequential()
model.add(Embedding(input_dim=10000, output_dim=100, input_length=100))
model.add(Conv1D(filters=128, kernel_size=5, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(units=len(label_encoder.classes_), activation='softmax'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


from sklearn.model_selection import train_test_split

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data_padded, labels, test_size=0.2, random_state=42)

# Train the model
model.fit(X_train, y_train, epochs=5, batch_size=32, validation_split=0.1)

# Evaluate the model performance on the test set
score, accuracy = model.evaluate(X_test, y_test)
print(f'Test score: {score}')
print(f'Test accuracy: {accuracy}')

# Epoch 1/5
# 4715/4715 [==============================] - 64s 13ms/step - loss: 2.3932 - accuracy: 0.3857 - val_loss: 2.1587 - val_accuracy: 0.4297
# Epoch 2/5
# 4715/4715 [==============================] - 74s 16ms/step - loss: 1.9283 - accuracy: 0.4773 - val_loss: 2.1182 - val_accuracy: 0.4428
# Epoch 3/5
# 4715/4715 [==============================] - 75s 16ms/step - loss: 1.6471 - accuracy: 0.5409 - val_loss: 2.2329 - val_accuracy: 0.4210
# Epoch 4/5
# 4715/4715 [==============================] - 65s 14ms/step - loss: 1.3519 - accuracy: 0.6189 - val_loss: 2.4508 - val_accuracy: 0.4092
# Epoch 5/5
# 4715/4715 [==============================] - 66s 14ms/step - loss: 1.0740 - accuracy: 0.7003 - val_loss: 2.7684 - val_accuracy: 0.3867
# 1310/1310 [==============================] - 4s 3ms/step - loss: 2.7648 - accuracy: 0.3867
# Test score: 2.7648091316223145
# Test accuracy: 0.38674652576446533

#Summary: While the model shows a strong ability to learn from the training data, as evidenced by decreasing training loss and increasing training accuracy, 
# it appears to overfit. Overfitting is demonstrated by increasing validation loss and decreasing validation accuracy in later epochs. 


Epoch 1/5
4715/4715 [==============================] - 64s 13ms/step - loss: 2.3932 - accuracy: 0.3857 - val_loss: 2.1587 - val_accuracy: 0.4297
Epoch 2/5
4715/4715 [==============================] - 74s 16ms/step - loss: 1.9283 - accuracy: 0.4773 - val_loss: 2.1182 - val_accuracy: 0.4428
Epoch 3/5
4715/4715 [==============================] - 75s 16ms/step - loss: 1.6471 - accuracy: 0.5409 - val_loss: 2.2329 - val_accuracy: 0.4210
Epoch 4/5
4715/4715 [==============================] - 65s 14ms/step - loss: 1.3519 - accuracy: 0.6189 - val_loss: 2.4508 - val_accuracy: 0.4092
Epoch 5/5
1310/1310 [==============================] - 4s 3ms/step - loss: 2.7648 - accuracy: 0.3867
Test score: 2.7648091316223145
Test accuracy: 0.38674652576446533
